<a href="https://colab.research.google.com/github/Nurochman79/DataScience/blob/main/HR_PREFORMANCE_PREDICTION_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# NGULIK MEMBUAT MODEL PREDIKSI PERFORMANCE KARYAWAN. HASILNYA YAITU BERDASRKAN BEBERAPA OPSI THRESHOLD DIPILIH NILAI THRESHOLD 0.55 YANG MENGHASILKAN  NILAI F SCORE REALISTIS YAITU 27% (PRESISION 17% DAN RECALL 60%)
# UNTUK PERFORMANCE HIGH DAN F1 SCORE 62% (PRESISSION 87% DAN RECALL 49%) UNTUK PERFORMANCE MEDIUM

In [1]:
# Persiapan modul yang dipakai
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix


In [2]:
# Load dataset
hr_data = pd.read_csv('/content/Pro3_HR_Performance_Predictive.csv', sep=',')

In [ ]:
# Memahami dataset

In [3]:
hr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [ ]:
# Mengelola leakage data dan missing value

In [4]:
hr_data['PerformanceRating'].value_counts()

,count
PerformanceRating,
3,1244
4,226


In [5]:
# Lakukan feature engineering untuk menghindari leakage data dengan membuat feature Performance _calss
hr_data["performance_class"] = np.where(
    hr_data["PerformanceRating"] >= 4,
    "High",
    "Medium"
)

In [6]:
hr_data = hr_data.drop(columns=["PerformanceRating"])


In [7]:
# Feature yang tidak related dengan target dihapus
leakage_cols = [
    "PercentSalaryHike",
    "StockOptionLevel",
    "YearsSinceLastPromotion",
    "TrainingTimesLastYear",
    "JobLevel",
    "MonthlyIncome",
    "TotalWorkingYears",
    "EmployeeNumber",
    "EmployeeCount",
    "StandardHours",
    "Over18",
     "YearsAtCompany",
    "YearsWithCurrManager"
]

leakage_cols_exist = [c for c in leakage_cols if c in hr_data.columns]

print("Kolom yang dihapus:", leakage_cols_exist)

hr_data = hr_data.drop(columns=leakage_cols_exist)

Kolom yang dihapus: ['PercentSalaryHike', 'StockOptionLevel', 'YearsSinceLastPromotion', 'TrainingTimesLastYear', 'JobLevel', 'MonthlyIncome', 'TotalWorkingYears', 'EmployeeNumber', 'EmployeeCount', 'StandardHours', 'Over18', 'YearsAtCompany', 'YearsWithCurrManager']


In [9]:
#Make sure no missing value
hr_data.isna().sum()

,0
Age,0
Attrition,0
BusinessTravel,0
DailyRate,0
Department,0
DistanceFromHome,0
Education,0
EducationField,0
EnvironmentSatisfaction,0
Gender,0


In [10]:
hr_data.isnull().sum()

,0
Age,0
Attrition,0
BusinessTravel,0
DailyRate,0
Department,0
DistanceFromHome,0
Education,0
EducationField,0
EnvironmentSatisfaction,0
Gender,0


In [ ]:
# Menentukan target dan feature

In [11]:
TARGET = "performance_class"

X = hr_data.drop(columns=[TARGET])
y = hr_data[TARGET]


In [12]:
hr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EnvironmentSatisfaction   1470 non-null   int64 
 9   Gender                    1470 non-null   object
 10  HourlyRate                1470 non-null   int64 
 11  JobInvolvement            1470 non-null   int64 
 12  JobRole                   1470 non-null   object
 13  JobSatisfaction           1470 non-null   int64 
 14  MaritalStatus           

In [ ]:
# Transform data kategori ke numerikal

In [13]:
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns


In [14]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)


In [ ]:
# Men-train dan split data (tidak perlu scalling data keran memakai Random Forest)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


In [ ]:
# Membuat model prediksi dengan RandomForest

In [16]:
rf_model2 = Pipeline(steps=[
    ("prep", preprocessor),
    ("model", RandomForestClassifier(
        n_estimators=300,
        max_depth=6,
        min_samples_leaf=20,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    ))
])

rf_model2.fit(X_train, y_train)


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  Index(['Age', 'DailyRate', 'DistanceFromHome', 'Education',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement',
       'JobSatisfaction', 'MonthlyRate', 'NumCompaniesWorked',
       'RelationshipSatisfaction', 'WorkLifeBalance', 'YearsInCurrentRole'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender',
       'JobRole', 'MaritalStatus', 'OverTime'],
      dtype='object'))])),
                ('model',
                 RandomForestClassifier(class_weight='balanced', max_depth=6,
                                        min_samples_leaf=20, n_estimators=300,
                                        n_jobs=-1, random_state=42))])

In [ ]:
# Men-tunning model untuk mendapat akurasi yang baik, tepat dan relevan (masuk akal dan jujur)

In [17]:
y_pred = rf_model2.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

        High       0.17      0.09      0.12        45
      Medium       0.85      0.92      0.88       249

    accuracy                           0.80       294
   macro avg       0.51      0.51      0.50       294
weighted avg       0.75      0.80      0.77       294



In [18]:
proba = rf_model2.predict_proba(X_test)

y_pred_custom = np.where(
    proba[:, 1] > 0.3,
    "High",
    "Medium"
)

print(classification_report(y_test, y_pred_custom))


              precision    recall  f1-score   support

        High       0.15      1.00      0.27        45
      Medium       0.00      0.00      0.00       249

    accuracy                           0.15       294
   macro avg       0.08      0.50      0.13       294
weighted avg       0.02      0.15      0.04       294



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
proba = rf_model2.predict_proba(X_test)

y_pred_custom = np.where(
    proba[:, 1] > 0.4,
    "High",
    "Medium"
)

print(classification_report(y_test, y_pred_custom))


              precision    recall  f1-score   support

        High       0.15      1.00      0.27        45
      Medium       0.00      0.00      0.00       249

    accuracy                           0.15       294
   macro avg       0.08      0.50      0.13       294
weighted avg       0.02      0.15      0.04       294



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
proba = rf_model2.predict_proba(X_test)

y_pred_custom = np.where(
    proba[:, 1] > 0.5,
    "High",
    "Medium"
)

print(classification_report(y_test, y_pred_custom))


              precision    recall  f1-score   support

        High       0.15      0.91      0.26        45
      Medium       0.83      0.08      0.14       249

    accuracy                           0.20       294
   macro avg       0.49      0.49      0.20       294
weighted avg       0.72      0.20      0.16       294



In [21]:
proba = rf_model2.predict_proba(X_test)

y_pred_custom = np.where(
    proba[:, 1] > 0.55,
    "High",
    "Medium"
)

print(classification_report(y_test, y_pred_custom))


              precision    recall  f1-score   support

        High       0.17      0.60      0.27        45
      Medium       0.87      0.49      0.62       249

    accuracy                           0.50       294
   macro avg       0.52      0.54      0.45       294
weighted avg       0.76      0.50      0.57       294



In [22]:
proba = rf_model2.predict_proba(X_test)

y_pred_custom = np.where(
    proba[:, 1] > 0.6,
    "High",
    "Medium"
)

print(classification_report(y_test, y_pred_custom))


              precision    recall  f1-score   support

        High       0.17      0.16      0.16        45
      Medium       0.85      0.87      0.86       249

    accuracy                           0.76       294
   macro avg       0.51      0.51      0.51       294
weighted avg       0.75      0.76      0.75       294



In [ ]:
# Develop model "Prediksi Performance Karyawan"

In [23]:
import joblib

joblib.dump(rf_model2, "rf_performance_model.pkl")
joblib.dump(X.columns.tolist(), "model_features.pkl")


['model_features.pkl']